# Modelová úloha - vzdálenosti mezi body v 3D
Zadání: pro N bodů v 3 rozměrném prostoru spočítejte vzájemnou vzdálenost $d$, která je pro dva body $x,y$ definovaná jako $\sqrt {\sum_{i=1}^3 {{{\left( {{x_i} - {y_i}} \right)}^2}} } $. Výslekem je tedy (symetrická) matice $N\times N$.

### Vstupní data
Začneme čistě v Pythonu, body reprezentujeme jako list touplů.

In [ ]:
# vygeneruje list bodů, tuplů (x, y, z)
import random
n = 200
points = [(random.random(), random.random(), random.random()) for i in range(n)]

In [ ]:
# vykreslí body
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter([p[0] for p in points], [p[1] for p in points], [p[2] for p in points])
plt.show()

### Prvnotní nástřel - vzdálenost mezi body

In [ ]:
import math

def dist_py1(points):
    distances_all = []
    for point_1 in points:
        distances = []
        for point_2 in points:
            tmp_sum = 0
            for k in range(3):
                rozdil = point_1[k] - point_2[k]
                tmp_sum += rozdil ** 2
            tmp_dist = math.sqrt(tmp_sum)
            distances.append(tmp_dist)
        distances_all.append(distances)

    return distances_all


In [ ]:
%time res1 = dist_py1(points)

In [ ]:
plt.imshow(res1)

### Profilování

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -u 1e-6 -f dist_py1 res1 = dist_py1(points)

## Optimalizace na úrovni algoritmu

Často je možné získat větší rychlost bez nutnosti optimalizace na úrovni kódu, ale lepším algoritmem. Mezi standardní příklady patří třídící algoritmy, kde různé algoritmy mají různou složitost. 

V našem případě si můžeme všimnout, že výsledek je symetrický, tedy pro všechny body platí, že vzdálenost od bodu $A$ k bodu $B$ je stejná jako vzdálenost od bodu $B$ k bodu $A$, navíc na diagonále je vždy nula.

Dále můžeme nahradit `x**2` za `x*x`, které bude pravděpodobně ryhlejší.

In [ ]:
def dist_py2(points):
    # nejdříve vyrobíme prázdný 2d list
    distances_all: list[list[float]] = [[0 for _ in range(len(points))] 
                                        for _ in range(len(points))]
    for i in range(len(points)):
        for j in range(i+1,len(points)): # vyplníme jen hodnoty nad diagonálou
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i][k] - points[j][k]
                tmp_sum += rozdil * rozdil
            tmp_dist = math.sqrt(tmp_sum)
            distances_all[i][j] = tmp_dist
            distances_all[j][i] = tmp_dist

    return distances_all

In [ ]:
%time res2 = dist_py2(points)

In [ ]:
# pro srovnání
%time res1 = dist_py1(points)

Raději zkontrolujeme, že počítáme stále to stejné:

In [ ]:
# zkontrolujeme jestli res1 a res2 jsou stejné pomocí numpy allclose
import numpy as np
print(np.allclose(res1, res2))

Podíváme se na profilování nové verze:

In [ ]:
%lprun -u 1e-6 -f dist_py2 res2 = dist_py2(points)

In [ ]:
# pro srovnání
%lprun -u 1e-6 -f dist_py1 res1 = dist_py1(points)

## Optimalizace pomocí vektorizace (NumPy)

Nejprve přepíšeme funkci tak, aby pracovala s NumPy polemi:

In [ ]:
import numpy as np
import math
def dist_np1(points):
    n = points.shape[0]
    distances_all = np.zeros((n, n))
    for i in range(n):
        for j in range(i+1, n):
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i, k] - points[j, k]
                tmp_sum += rozdil*rozdil
            tmp_dist = math.sqrt(tmp_sum)
            distances_all[i, j] = tmp_dist
            distances_all[j, i] = tmp_dist

    return distances_all

In [ ]:
points_np = np.array(points)

In [ ]:
%time res3 = dist_np1(points_np)

Tohle vůbec nepomohlo, ba naopak!

Důvodem je fakt, že `numpy` je rozhraní do implementace v nízkoúrovňovém jazyce C/Fortran, které se však zde volá stále dokola (v cyklech), a tedy nasbíráme obrovský overhead.

In [ ]:
# pro jistotu overíme, že výsledky jsou stejné
print(np.allclose(res1, res3))

## Vektorizace - zbavme se vnitřních smyček

- pokusíme se zbavit vnitřních cyklů

In [ ]:
import numpy as np
import math
def dist_np2(points):
    n = points.shape[0]
    distances_all = np.zeros((n, n))
    for i in range(n):
        rozdily = points[i, :] - points[i+1: n, :]
        tmp_sum = np.sum(rozdily * rozdily, axis=1)
        tmp_dist = np.sqrt(tmp_sum)
        distances_all[i, i+1: n] = tmp_dist
        distances_all[i+1: n, i] = tmp_dist

    return distances_all

In [ ]:
%time res4 = dist_np2(points_np)

To už je je mnohem lepší! 

Raději zkontrolujeme, že počítáme stále to stejné:

In [ ]:
# kontrola
print(np.allclose(res1, res4))

In [ ]:
# jak vypadá profilování?
%lprun -u 1e-6 -f dist_np2 res4 = dist_np2(points_np)

## Úplná vektorizace na úkor redundance výpočtů

Vrátíme se zpět k původnímu kódu ale zkusíme vektorizovat vše, tedy ať nepoužíváme žádné cykly.

In [ ]:
import numpy as np
import math
def dist_np3(points):
    # Použijeme broadcasting a spočítáme všechny rozdíly najednou
    diffs = points[:, None, :] - points # shape (n, n, 3)
    
    # spočteme druhou mocninu všech rozdílů a sečteme skrz poslední osu
    tmp_sum = np.sum(diffs*diffs, axis=-1)

    distances_all = np.sqrt(tmp_sum)
    return distances_all

In [ ]:
%time res5 = dist_np3(points_np)

In [ ]:
# kontrola
print(np.allclose(res1, res5))

Zdá se, že poslední redundantní verze je nejrychlejší!

Teď je nařadě benchmarkování. Skutečně je to nejrychlejší vždy?

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(4, 12)]
time_py1 = []
time_py2 = []
time_np2 = []
time_np3 = []

for n in velikosti:
    points = [(random.random(), random.random(), random.random()) for i in range(n)]
    points_np = np.array(points)
    start = time.time()
    res1 = dist_py1(points)
    time_py1.append(time.time() - start)
    start = time.time()
    res2 = dist_py2(points)
    time_py2.append(time.time() - start)
    start = time.time()
    res3 = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    res4 = dist_np3(points_np)
    time_np3.append(time.time() - start)

    print(n, time_py1[-1], time_py2[-1], time_np2[-1], time_np3[-1])

# log-log grafy
plt.loglog(velikosti, time_py1, label="py1")
plt.loglog(velikosti, time_py2, label="py2")
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np3, label="np3")
plt.legend()

## Jak si povede Numba?

Nejlepší byl zatím kód `dist_np2`, zkusíme jej "zakomplilovat" pomocí Numby.

In [ ]:
from numba import jit
dist_np2_numba = jit(dist_np2, nopython=True)

In [ ]:
# srovnání dist_np2 a dist_np2_numba
points_np = np.random.rand(1000, 3)
res1 = dist_np2(points_np)
res2 = dist_np2_numba(points_np)
print(np.allclose(res1, res2))

Při testování se kód zároveň kompiluje což je třeba mít na mysli při měření času.

In [ ]:
%timeit res1 = dist_np2(points_np)

In [ ]:
%timeit res2 = dist_np2_numba(points_np)

Zkusíme ještě původní verzi se smyčkami `dist_np1`:

In [ ]:
dist_np1_numba = jit(dist_np1, nopython=True)

In [ ]:
res1 = dist_np2(points_np)
res2 = dist_np1_numba(points_np)
print(np.allclose(res1, res2))

In [ ]:
%timeit res2 = dist_np1_numba(points_np)

Obojí výrazně zrychlilo běh, a kupodivu `dist_np1_numba` je ještě rychlejší než `dist_np2_numba`!

Toto je důsledek toho, že Numba si sama překládá kód do C a optimalizuje. Užití vektorizace jí vnucuje jeden konktrétní způsob, který nemusí být vždy nejlepší (jako je to zde).

Přidáme ji do benchmarku:

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(6, 14)]
time_np2 = []
time_np1_numba = []
time_np2_numba = []

for n in velikosti:
    points = [(random.random(), random.random(), random.random()) for i in range(n)]
    points_np = np.array(points)
    start = time.time()
    _ = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    _ = dist_np1_numba(points_np)
    time_np1_numba.append(time.time() - start)
    start = time.time()
    _ = dist_np2_numba(points_np)
    time_np2_numba.append(time.time() - start)

    print(n, time_np2[-1], time_np1_numba[-1], time_np2_numba[-1])

# log-log grafy
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np1_numba, label="np1_numba")
plt.loglog(velikosti, time_np2_numba, label="np2_numba")
plt.legend()

## A co Cython?

In [ ]:
%load_ext Cython

In [ ]:
%%cython --compile-args=-O3

import numpy as np
cimport numpy as np
from libc.math cimport sqrt
cimport cython

ctypedef np.float64_t DTYPE_t

@cython.boundscheck(False)
@cython.wraparound(False)
def dist_cython(np.ndarray[DTYPE_t, ndim=2] points):
    cdef int n = points.shape[0]
    cdef np.ndarray[DTYPE_t, ndim=2] distances_all = np.zeros((n, n), dtype=np.float64)
    cdef int i, j, k
    cdef double tmp_sum, rozdil, tmp_dist

    for i in range(n):
        for j in range(i+1, n):
            tmp_sum = 0
            for k in range(3):
                rozdil = points[i, k] - points[j, k]
                tmp_sum += rozdil * rozdil
            tmp_dist = sqrt(tmp_sum)
            distances_all[i, j] = tmp_dist
            distances_all[j, i] = tmp_dist

    return distances_all


In [ ]:
# vyzkoušíme si, jestli to funguje
points_np = np.random.rand(1000, 3)
res5 = dist_np2(points_np)
res6 = dist_cython(points_np)
np.allclose(res5, res6)

In [ ]:
# časové měření
%timeit res6 = dist_cython(points_np)

In [ ]:
# porovnání s numba
%timeit res5 = dist_np1_numba(points_np)

## Porovnání nejlepších variant

In [ ]:
import time
import matplotlib.pyplot as plt

velikosti =[2**i for i in range(6, 14)]
time_np2 = []
time_np1_numba = []
time_cython = []

for n in velikosti:
    points_np = np.random.rand(n, 3)
    start = time.time()
    _ = dist_np2(points_np)
    time_np2.append(time.time() - start)
    start = time.time()
    _ = dist_np1_numba(points_np)
    time_np1_numba.append(time.time() - start)
    start = time.time()
    _ = dist_cython(points_np)
    time_cython.append(time.time() - start)

    print(n, time_np2[-1], time_np1_numba[-1], time_cython[-1])

# log-log grafy
plt.loglog(velikosti, time_np2, label="np2")
plt.loglog(velikosti, time_np1_numba, label="np1_numba")
plt.loglog(velikosti, time_cython, label="cython")
plt.legend()